In [5]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/vco/Projects/pm4py-dcr
/home/vco/Projects/pm4py-dcr


In [6]:
import pm4py
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.discovery.dcr_discover import algorithm as dcr_alg

# File names

In [7]:
event_log_file = '/home/vco/Datasets/Sepsis Cases - Event Log.xes'
dcrxml_file = 'models/sepsis_dcr.xml'
dcr_title = 'Sepsis Cases'

# Mine DCR graph using the DisCoveR miner

In [8]:
event_log = pm4py.read_xes(event_log_file,return_legacy_log_object=True)

/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 1830.54it/s]


In [12]:
dcr_graph = dcr_alg.apply(input_log=event_log,variant=dcr_alg.DCR_BASIC,findAdditionalConditions=True) # this will need updating to the new pm4py standards (for now it works)

/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [54]:
for k,v in dcr_graph.items():
    print(k,v)

events {'Release D', 'Release E', 'Release B', 'Return ER', 'ER Registration', 'Admission IC', 'CRP', 'IV Liquid', 'Admission NC', 'LacticAcid', 'Release A', 'Leucocytes', 'ER Sepsis Triage', 'ER Triage', 'IV Antibiotics', 'Release C'}
conditionsFor {'Release D': {'Admission NC', 'ER Sepsis Triage', 'Leucocytes', 'CRP'}, 'Release E': {'Admission NC', 'ER Sepsis Triage', 'Leucocytes', 'CRP'}, 'Release B': {'ER Triage', 'CRP', 'Leucocytes', 'ER Sepsis Triage', 'ER Registration'}, 'Return ER': {'Release D', 'Release E', 'Admission IC', 'Release A', 'Release C'}, 'ER Registration': set(), 'Admission IC': {'ER Triage', 'ER Registration'}, 'CRP': set(), 'IV Liquid': set(), 'Admission NC': {'ER Triage', 'ER Registration'}, 'LacticAcid': set(), 'Release A': {'ER Triage', 'ER Registration'}, 'Leucocytes': set(), 'ER Sepsis Triage': set(), 'ER Triage': set(), 'IV Antibiotics': {'ER Sepsis Triage'}, 'Release C': {'Admission NC', 'ER Sepsis Triage', 'Leucocytes', 'CRP'}}
milestonesFor {'Release D'

# Write the graph to file

In [15]:
dcr_exporter.apply(dcr_graph=dcr_graph,
                    path=dcrxml_file,
                    variant = dcr_exporter.DCR_XML_SIMPLE,
                    dcr_title=dcr_title)

# Read the dcrxml graph file
Works also with graphs downlaoded from the dcrgraphs.net portal

In [56]:
dcr_graph_loaded = dcr_importer.apply(dcrxml_file)
print(dcr_graph_loaded)

{'events': {'ERSepsisTriage', 'Leucocytes', 'ReleaseC', 'IVLiquid', 'ReturnER', 'ReleaseE', 'ERTriage', 'AdmissionIC', 'ReleaseD', 'ReleaseB', 'ERRegistration', 'CRP', 'IVAntibiotics', 'ReleaseA', 'AdmissionNC', 'LacticAcid'}, 'labels': {'ERSepsisTriage', 'Leucocytes', 'ReleaseC', 'IVLiquid', 'ReturnER', 'ReleaseE', 'ERTriage', 'AdmissionIC', 'ReleaseD', 'ReleaseB', 'ERRegistration', 'CRP', 'IVAntibiotics', 'ReleaseA', 'AdmissionNC', 'LacticAcid'}, 'labelMapping': set(), 'conditionsFor': {'ReleaseD': {'ERSepsisTriage', 'AdmissionNC', 'Leucocytes', 'CRP'}, 'ReleaseE': {'ERSepsisTriage', 'AdmissionNC', 'Leucocytes', 'CRP'}, 'ReleaseB': {'ERSepsisTriage', 'Leucocytes', 'ERTriage', 'ERRegistration', 'CRP'}, 'ReturnER': {'ReleaseC', 'ReleaseE', 'AdmissionIC', 'ReleaseD', 'ReleaseA'}, 'AdmissionIC': {'ERTriage', 'ERRegistration'}, 'AdmissionNC': {'ERTriage', 'ERRegistration'}, 'ReleaseA': {'ERTriage', 'ERRegistration'}, 'IVAntibiotics': {'ERSepsisTriage'}, 'ReleaseC': {'ERSepsisTriage', 'Adm

# Play with the execution semantics (note milestones and no-responses are not supported yet)

In [63]:
dcr = dcr_graph_loaded
print(dcr['events']) # all the events

{'ERSepsisTriage', 'Leucocytes', 'ReleaseC', 'IVLiquid', 'ReturnER', 'ReleaseE', 'ERTriage', 'AdmissionIC', 'ReleaseD', 'ReleaseB', 'ERRegistration', 'CRP', 'IVAntibiotics', 'ReleaseA', 'AdmissionNC', 'LacticAcid'}


In [64]:
print(dcr_semantics.enabled(dcr)) # check enabled events

{'ERSepsisTriage', 'CRP', 'Leucocytes', 'ERTriage', 'ERRegistration', 'IVLiquid', 'LacticAcid'}


In [65]:
event_to_execute = 'ER Registration' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

[!] Event ER Registration does not exist!
{'ERSepsisTriage', 'CRP', 'Leucocytes', 'ERTriage', 'ERRegistration', 'IVLiquid', 'LacticAcid'}


In [62]:
event_to_execute = 'ER Sepsis Triage' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

[!] Event ER Sepsis Triage not enabled!
{'Leucocytes', 'Admission NC', 'ER Triage', 'Admission IC', 'IV Antibiotics', 'CRP', 'IV Liquid', 'LacticAcid', 'Release A'}


In [52]:
dcr_semantics.is_accepting(dcr) # this checks the acceptance criteria for the dcr graph

True

In [33]:
dcr['marking']['pending']

set()